<a href="https://colab.research.google.com/github/earltreloar/logosfield-cddr-analysis/blob/main/mechanism_1_and_2_alignment_density_per_logosfield.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import hashlib, json, platform, numpy as np, pandas as pd, sys
from datetime import datetime
def sha256(path, block=1<<20):
    h=hashlib.sha256()
    with open(path,'rb') as f:
        while True:
            b=f.read(block)
            if not b: break
            h.update(b)
    return h.hexdigest()

RUN_META = {
    "timestamp_utc": datetime.utcnow().isoformat()+"Z",
    "python": sys.version.split()[0],
    "platform": platform.platform(),
    "numpy": np.__version__,
    "pandas": pd.__version__,
    # fill these with your actual filenames
    "files": {
        "spins_csv": "spin3.csv",
        "nodes_csv": "nodes.csv"
    }
}
for k,p in RUN_META["files"].items():
    try: RUN_META["files"][k] = {"path": p, "sha256": sha256(p)}
    except: RUN_META["files"][k] = {"path": p, "sha256": None}

print(json.dumps(RUN_META, indent=2))


{
  "timestamp_utc": "2025-08-30T03:47:27.914674Z",
  "python": "3.12.11",
  "platform": "Linux-6.1.123+-x86_64-with-glibc2.35",
  "numpy": "2.0.2",
  "pandas": "2.2.2",
  "files": {
    "spins_csv": {
      "path": "spin3.csv",
      "sha256": "410d14cbdd2f83090a68191f0392faac77ebc21cc5dc3378051aa4b8b4db5974"
    },
    "nodes_csv": {
      "path": "nodes.csv",
      "sha256": "3e6f65c65cbc6a98f21aa0dea484709e3430fcb0ca3293a954e4931c41c02368"
    }
  }
}


/tmp/ipython-input-1315706433.py:13: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp_utc": datetime.utcnow().isoformat()+"Z",


In [7]:
# ===== Mechanism 1 — Scalar–Spin Style (explicit upload prompts) =====
# Needs: spins CSV (with x,y,z,sx,sy,sz), nodes CSV (ra,dec[,weight]); optional density*.csv
import os, zipfile, numpy as np, pandas as pd
from scipy.stats import binomtest

try:
    from google.colab import files
    IN_COLAB = True
except:
    IN_COLAB = False

def read_csv(path):
    try:    return pd.read_csv(path)
    except: return pd.read_csv(path, sep=None, engine="python")

def auto_unzip():
    for f in list(os.listdir(".")):
        if f.lower().endswith(".zip"):
            with zipfile.ZipFile(f,"r") as zf:
                zf.extractall(".")
                print(f"📦 Extracted {f} -> {zf.namelist()}")

def pick_file(keyword_list, exclude_keywords=("node","dens"), required=True, label="file"):
    def finder():
        cands=[]
        for f in os.listdir("."):
            if not f.lower().endswith(".csv"): continue
            low=f.lower()
            if any(ex in low for ex in exclude_keywords): continue
            if any(k in low for k in keyword_list): cands.append(f)
        return cands[0] if cands else None

    f = finder()
    while required and f is None and IN_COLAB:
        print(f"📤 Please upload the {label} (CSV or ZIP).")
        up = files.upload()
        for n,b in up.items():
            with open(n,"wb") as fh: fh.write(b)
        auto_unzip()
        f = finder()

    if required and f is None:
        raise FileNotFoundError(f"Missing required {label}.")
    return f

# 1) Initial generic upload (optional; you can skip and it will prompt later)
if IN_COLAB:
    print("📤 You can upload now (or wait for prompts): spins, nodes, optional density")
    up0 = files.upload()
    for n,b in up0.items():
        with open(n,"wb") as fh: fh.write(b)
auto_unzip()

# 2) Explicitly collect required files
# spins: must contain x,y,z,sx,sy,sz
sp_path    = pick_file(["spin","zoo","hsc","jwst","ceers","jades","galaxy","table2"], label="SPINS file")
# nodes: must contain ra/dec (headers like ra/dec or RAJ2000/DEJ2000 etc.)
nodes_path = None
def find_nodes():
    for f in os.listdir("."):
        if f.lower().endswith(".csv") and "node" in f.lower(): return f
    return None
nodes_path = find_nodes()
while nodes_path is None and IN_COLAB:
    print("📤 Please upload the NODES file (CSV or ZIP; headers ra/dec[,weight]).")
    up = files.upload()
    for n,b in up.items():
        with open(n,"wb") as fh: fh.write(b)
    auto_unzip()
    nodes_path = find_nodes()
if nodes_path is None:
    raise FileNotFoundError("Missing required nodes file (e.g., 'nodes.csv').")

# density (optional)
dens_path = None
for f in os.listdir("."):
    if f.lower().endswith(".csv") and "dens" in f.lower():
        dens_path = f; break

print(f"🧾 Using spins:   {sp_path}")
print(f"🧾 Using nodes:   {nodes_path}")
if dens_path: print(f"🧾 Using density: {dens_path}")

# 3) Load and validate columns
sp    = read_csv(sp_path)
nodes = read_csv(nodes_path)

# spins need x,y,z,sx,sy,sz (case-insensitive)
need = {"x","y","z","sx","sy","sz"}
have = {c.lower(): c for c in sp.columns}
missing = [c for c in need if c not in have]
if missing:
    raise ValueError(f"Spins file must contain columns {sorted(list(need))} (found {list(sp.columns)}).")

v  = sp[[have["x"], have["y"], have["z"]]].to_numpy(float)
sv = sp[[have["sx"], have["sy"], have["sz"]]].to_numpy(float)
sv = sv / np.clip(np.linalg.norm(sv, axis=1, keepdims=True), 1e-12, None)

# nodes need RA/Dec (accept aliases)
def pick(df, cands):
    for c in df.columns:
        if c.lower() in cands: return c
    return None
ra_col  = pick(nodes, {"ra","ra_deg","raj2000","alpha","alphaj2000","radeg"})
dec_col = pick(nodes, {"dec","dec_deg","dej2000","delta","deltaj2000","decdeg"})
if ra_col is None or dec_col is None:
    raise ValueError(f"Nodes file must have RA/Dec columns. Found: {list(nodes.columns)}")

def radec_to_unit(ra_deg, dec_deg):
    ra = np.radians(pd.to_numeric(ra_deg, errors="coerce") % 360.0)
    dec= np.radians(pd.to_numeric(dec_deg, errors="coerce").clip(-90,90))
    x = np.cos(dec)*np.cos(ra); y = np.cos(dec)*np.sin(ra); z = np.sin(dec)
    return np.stack([x,y,z], axis=1)

g_nodes = radec_to_unit(nodes[ra_col], nodes[dec_col])

# 4) Local direction via nearest node (3D kNN on unit vectors)
from sklearn.neighbors import KDTree
tree = KDTree(g_nodes)
idx  = tree.query(v, k=1)[1][:,0]
g    = g_nodes[idx]

# 5) Scalar–spin alignment
obs_spin = np.sign(np.sum(sv * g, axis=1)).astype(int); obs_spin[obs_spin==0] = 1
pred     = np.sign(np.sum(g * np.array([0,0,1.0]), axis=1)).astype(int); pred[pred==0] = 1

aligned = (obs_spin == pred)
N = aligned.size; k = int(aligned.sum()); frac = k/max(N,1)
p = binomtest(k, N, 0.5, alternative="greater").pvalue
ci = binomtest(k, N, 0.5, alternative="greater").proportion_ci(0.95)

print("\n✅ Scalar–spin result")
print(f"  N={N}  aligned={frac:.4f}  p={p:.3g}  95% CI=[{ci.low:.4f},{ci.high:.4f}]")


📤 You can upload now (or wait for prompts): spins, nodes, optional density


Saving nodes (3).csv to nodes (3) (5).csv
Saving spin3.csv to spin3 (2).csv
Saving density2.csv to density2 (5).csv
📦 Extracted GalaxyZoo1_DR_table2.csv (1).zip -> ['GalaxyZoo1_DR_table2.csv']
📦 Extracted GalaxyZoo1_DR_table2.csv.zip -> ['GalaxyZoo1_DR_table2.csv']
🧾 Using spins:   spin3 (1).csv
🧾 Using nodes:   nodes (3).csv
🧾 Using density: density2 (2).csv

✅ Scalar–spin result
  N=500  aligned=0.4780  p=0.848  95% CI=[0.4404,1.0000]


In [3]:
!pip install healpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 43.8 MB/s eta 0:00:00


In [10]:
import pandas as pd
from glob import glob

# Load all candidate CSVs in /content
files = glob("/content/*.csv") + glob("/content/*.csv.gz") + glob("/content/*.xlsx")

print("📂 Checking all uploaded table files...\n")

for f in files:
    print(f"\n➡️  File: {f}")
    try:
        df = pd.read_csv(f) if f.endswith('.csv') or f.endswith('.csv.gz') else pd.read_excel(f)
        print("✅ Columns:", list(df.columns))
        missing = [col for col in ['ra', 'dec', 'spin'] if col not in df.columns]
        if missing:
            print("❌ Missing columns:", missing)
        else:
            print("✅ All required columns present.")
    except Exception as e:
        print("⚠️ Failed to read:", e)


📂 Checking all uploaded table files...


➡️  File: /content/data_hsc.csv
✅ Columns: ['RA', 'Dec', 'z', 'direction_cw_ccw']
❌ Missing columns: ['ra', 'dec', 'spin']

➡️  File: /content/GalaxyZoo1_DR_table2.csv.gz
✅ Columns: ['OBJID', 'RA', 'DEC', 'NVOTE', 'P_EL', 'P_CW', 'P_ACW', 'P_EDGE', 'P_DK', 'P_MG', 'P_CS', 'P_EL_DEBIASED', 'P_CS_DEBIASED', 'SPIRAL', 'ELLIPTICAL', 'UNCERTAIN']
❌ Missing columns: ['ra', 'dec', 'spin']

➡️  File: /content/HSC_STANDARDIZED copy.xlsx
✅ Columns: ['ra', 'dec', 'spin']
✅ All required columns present.

➡️  File: /content/master_highz_plus_goodsn_filled copy (1)_STANDARDIZED (1) (1).xlsx
✅ Columns: ['ra', 'dec', 'spin', 'p_cw', 'p_ccw']
✅ All required columns present.

➡️  File: /content/shamir_jades_proxy_zge10.xlsx
✅ Columns: ['object_id', 'spin', 'dataset', 'z']
❌ Missing columns: ['ra', 'dec']


In [14]:
# --- Upload prompt ---
from google.colab import files
import pandas as pd
import numpy as np

uploaded = files.upload()

# --- Identify and load uploaded files ---
galaxy_file = None
logosfield_file = None

for f in uploaded:
    if f.endswith(('.csv', '.csv.gz', '.xlsx')) and galaxy_file is None:
        galaxy_file = f
    elif f.endswith('.npy') and logosfield_file is None:
        logosfield_file = f

if galaxy_file is None:
    raise ValueError("❌ No galaxy file (.csv/.xlsx) found.")
if logosfield_file is None:
    raise ValueError("❌ No Logosfield .npy map file found.")

# --- Load galaxy data ---
try:
    if galaxy_file.endswith('.xlsx'):
        df = pd.read_excel(galaxy_file)
    else:
        df = pd.read_csv(galaxy_file)
except Exception as e:
    raise RuntimeError(f"❌ Failed to read galaxy file: {e}")

# --- Check required columns ---
required_cols = ['ra', 'dec', 'spin']
missing_cols = [col for col in required_cols if col not in df.columns]
if missing_cols:
    raise ValueError(f"❌ Galaxy file missing columns: {missing_cols}")
else:
    print(f"✅ Loaded galaxy file: {galaxy_file}")
    print(f"  Columns: {list(df.columns)}")
    print(f"✅ All required columns present: {required_cols}")

# --- Load Logosfield map ---
try:
    logosfield_map = np.load(logosfield_file, allow_pickle=True)
    print(f"✅ Loaded Logosfield scalar map: {logosfield_file}")
except Exception as e:
    raise RuntimeError(f"❌ Failed to load Logosfield .npy file: {e}")


Saving master_highz_plus_goodsn_filled copy (1)_STANDARDIZED (1) (1).xlsx to master_highz_plus_goodsn_filled copy (1)_STANDARDIZED (1) (1) (4).xlsx
Saving HSC_STANDARDIZED copy.xlsx to HSC_STANDARDIZED copy (4).xlsx
Saving glimpse_mask.fits to glimpse_mask.fits
Saving Logosfield_scalar_density_map.npy to Logosfield_scalar_density_map (5).npy
✅ Loaded galaxy file: master_highz_plus_goodsn_filled copy (1)_STANDARDIZED (1) (1) (4).xlsx
  Columns: ['ra', 'dec', 'spin', 'p_cw', 'p_ccw']
✅ All required columns present: ['ra', 'dec', 'spin']
✅ Loaded Logosfield scalar map: Logosfield_scalar_density_map (5).npy


In [22]:
import numpy as np

try:
    path = "Logosfield_scalar_density_map.npy"
    data = np.load(path, allow_pickle=True)

    print(f"Loaded type: {type(data)}")
    if isinstance(data, np.ndarray):
        print(f"Array shape: {data.shape}")
        if data.ndim == 1:
            print("✅ Valid 1D HEALPix map.")
        elif data.ndim == 0:
            print("⚠️ Zero-dimensional array. Inspecting contents...")
            obj = data.item()
            print(f"Item type: {type(obj)}")
            if isinstance(obj, np.ndarray):
                print(f"Recovered array shape: {obj.shape}")
                if obj.ndim == 1:
                    print("✅ Extracted valid 1D HEALPix map. Re-saving...")
                    np.save("Logosfield_scalar_density_map_FIXED.npy", obj)
                else:
                    print("❌ Extracted array is not 1D. Manual inspection needed.")
            else:
                print("❌ Object inside .npy is not an ndarray.")
        else:
            print("❌ Map is not 1D. Reshape or fix required.")
    else:
        print("❌ File content is not a NumPy array.")

except Exception as e:
    print("❌ Error loading .npy:", type(e).__name__, "-", str(e))


Loaded type: <class 'numpy.ndarray'>
Array shape: ()
⚠️ Zero-dimensional array. Inspecting contents...
Item type: <class 'dict'>
❌ Object inside .npy is not an ndarray.


In [26]:
alm = hp.map2alm(smoothed_map)
grad_maps = hp.alm2map_der1(alm, nside=nside)

if grad_maps.shape[0] >= 2:
    dtheta_map = grad_maps[0]
    dphi_map = grad_maps[1]
    print("✅ Gradient maps computed successfully.")
    np.save("Logosfield_dtheta_map.npy", dtheta_map)
    np.save("Logosfield_dphi_map.npy", dphi_map)
    print("💾 Saved: Logosfield_dtheta_map.npy and Logosfield_dphi_map.npy")
else:
    raise ValueError("Unexpected gradient output shape:", grad_maps.shape)


✅ Gradient maps computed successfully.
💾 Saved: Logosfield_dtheta_map.npy and Logosfield_dphi_map.npy


In [23]:
import numpy as np

# Replace with uploaded file path
map_filename = "Logosfield_scalar_density_map.npy"

try:
    # Load the object (dict inside zero-dimensional ndarray)
    raw = np.load(map_filename, allow_pickle=True)
    print("🔍 Loaded type:", type(raw))
    print("Array shape:", raw.shape)

    if raw.ndim == 0:
        print("⚠️ Zero-dimensional array. Inspecting contents...")
        obj = raw.item()
        print("Item type:", type(obj))

        if isinstance(obj, dict):
            # Attempt to extract the first array from dict
            for key, val in obj.items():
                if isinstance(val, np.ndarray) and val.ndim == 1:
                    print(f"✅ Extracted key '{key}' with shape {val.shape}")
                    np.save("Logosfield_scalar_density_map_FIXED.npy", val)
                    print("💾 Saved as Logosfield_scalar_density_map_FIXED.npy")
                    break
            else:
                print("❌ No 1D ndarray found inside the dictionary.")
        else:
            print("❌ Top-level object is not a dictionary.")
    else:
        print("❌ Unexpected array shape — manual inspection needed.")

except Exception as e:
    print("❌ Error occurred:", type(e).__name__, "-", str(e))


🔍 Loaded type: <class 'numpy.ndarray'>
Array shape: ()
⚠️ Zero-dimensional array. Inspecting contents...
Item type: <class 'dict'>
✅ Extracted key 'map' with shape (196608,)
💾 Saved as Logosfield_scalar_density_map_FIXED.npy


In [35]:
# ==== Mechanism 1 · Option A (Galaxy Zoo only) — AUTO-DISCOVER + SEXAGESIMAL-SAFE ====
import os, sys, json, math, zipfile, random, glob
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt

THETA_GATE_DEG = 15.0
MIN_MARGIN = 0.05
ROTATIONS_DEG = [0,30,60,90]
MAX_SHUFFLE_N = 200
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED)

# --------- Deps ---------
def ensure_healpy_astropy():
    try:
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "healpy", "astropy"])
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
hp, SkyCoord, u = ensure_healpy_astropy()

# --------- Helpers ---------
def infer_nside(vec):
    n = int(vec.size); ns = int(round((n/12.0)**0.5))
    if 12*(ns**2) != n: raise ValueError(f"Map length {n} not 12*nside^2.")
    return ns

def autodiscover(patterns, roots=("/mnt/data","/content",".")):
    cands=[]
    for r in roots:
        if not os.path.isdir(r): continue
        for pat in patterns:
            cands += glob.glob(os.path.join(r, pat))
    cands = sorted(set(cands), key=lambda p: (0 if "galaxyzoo" in p.lower() else 1, len(os.path.basename(p))))
    return cands[0] if cands else None

def discover_gz1():
    return autodiscover([
        "GalaxyZoo*table2*.csv*","GalaxyZoo*DR*table2*.csv*","*GZ*table2*.csv*","*galaxy*zoo*table2*.csv*"
    ])

def discover_map(kind):  # 'dtheta'|'dphi'
    return autodiscover([f"*{kind}*.npy", f"*Logosfield*{kind}*.npy", f"*{kind}_map*.npy", f"*{kind} map*.npy"])

def discover_mask():
    return autodiscover(["*mask*.fits"])

def detect_ra_dec_columns(df):
    L = {c.lower(): c for c in df.columns}
    ra = next((L[k] for k in ("ra","ra_deg","ra (deg)","ra_deg_j2000","ra_j2000") if k in L),
              next((c for c in df.columns if "ra" in c.lower()), None))
    dec = next((L[k] for k in ("dec","dec_deg","dec (deg)","dec_deg_j2000","dec_j2000","de") if k in L),
               next((c for c in df.columns if "dec" in c.lower() or c.lower()=="de"), None))
    if ra is None or dec is None: raise ValueError("RA/Dec columns not found.")
    return ra, dec

def parse_ra_dec_mixed(ra_series, dec_series):
    """
    Returns (ra_deg, dec_deg) handling degrees or sexagesimal strings like '00:00:00.74' / '+12:34:56.7'.
    Uses astropy SkyCoord; falls back to numeric if already deg.
    """
    # Quick path: try numeric
    ra_num  = pd.to_numeric(ra_series,  errors="coerce")
    dec_num = pd.to_numeric(dec_series, errors="coerce")
    if ra_num.notna().all() and dec_num.notna().all():
        return ra_num.values.astype(float), dec_num.values.astype(float)

    # Sexagesimal or mixed: build strings and parse via SkyCoord
    ra_str  = ra_series.astype(str).str.strip()
    dec_str = dec_series.astype(str).str.strip()

    # Heuristic: if many have ":" or " " treat as sexagesimal
    if (ra_str.str.contains(":").mean() > 0.05) or (dec_str.str.contains(":").mean() > 0.05):
        sc = SkyCoord(ra=ra_str.values, dec=dec_str.values, unit=(u.hourangle, u.deg), frame="icrs")
        return sc.ra.deg.astype(float), sc.dec.deg.astype(float)

    # Otherwise, last try: assume degrees but with stray strings
    if ra_num.isna().any() or dec_num.isna().any():
        # Attempt flexible parse row-by-row with SkyCoord accepting deg strings
        out_ra, out_dec = [], []
        for r, d in zip(ra_series, dec_series):
            try:
                rnum = float(r); dnum = float(d)
                out_ra.append(rnum); out_dec.append(dnum); continue
            except Exception:
                try:
                    sc = SkyCoord(str(r), str(d), unit=(u.deg, u.deg), frame="icrs")
                    out_ra.append(float(sc.ra.deg)); out_dec.append(float(sc.dec.deg))
                except Exception:
                    out_ra.append(np.nan); out_dec.append(np.nan)
        return np.array(out_ra), np.array(out_dec)

    return ra_num.values.astype(float), dec_num.values.astype(float)

def extract_spin_gz1(df):
    p_cw = next((c for c in df.columns if c.lower() in ("p_cw","p(cw)","p_cw_prob","prob_cw")), None)
    p_acw = next((c for c in df.columns if c.lower() in ("p_acw","p(ccw)","p_acw_prob","prob_acw","p_ccw","prob_ccw")), None)
    if p_cw and p_acw:
        pc = pd.to_numeric(df[p_cw], errors="coerce").astype(float)
        pa = pd.to_numeric(df[p_acw], errors="coerce").astype(float)
        keep = (pc - pa).abs() >= MIN_MARGIN
        spin = np.where(pc > pa, 1, -1).astype(int)
        return pd.Series(spin, index=df.index), keep
    for c in df.columns:
        if c.lower() in ("spin","handedness","spiral","cw_ccw"):
            vals = df[c].astype(str).str.lower().str.strip()
            spin = np.where(vals.isin(["cw","+1","1","clockwise"]), 1,
                            np.where(vals.isin(["ccw","-1","counterclockwise","anticlockwise"]), -1, np.nan))
            keep = ~np.isnan(spin)
            return pd.Series(spin, index=df.index).astype(int), keep
    cw_flag = next((c for c in df.columns if "cw" in c.lower() and "flag" in c.lower()), None)
    ccw_flag = next((c for c in df.columns if "ccw" in c.lower() and "flag" in c.lower()), None)
    if cw_flag and ccw_flag:
        spin = np.where(df[cw_flag].astype(int)==1, 1,
                        np.where(df[ccw_flag].astype(int)==1, -1, np.nan))
        keep = ~np.isnan(spin)
        return pd.Series(spin, index=df.index).astype(int), keep
    raise ValueError("Could not find spins (P_CW/P_ACW or spin labels).")

def try_load_mask(mask_path, nside_maps):
    try:
        from astropy.io import fits
    except Exception:
        print("astropy not present; continuing without mask.")
        return None
    if not mask_path or not os.path.exists(mask_path): return None
    try:
        with fits.open(mask_path) as hdul:
            data = hdul[1].data if len(hdul)>1 else hdul[0].data
            vec = np.array(data).astype(float).ravel()
        ns = infer_nside(vec)
        return vec if ns==nside_maps else hp.ud_grade(vec, nside_maps, power=-2)
    except Exception as e:
        print(f"Mask load/resample failed: {e}; proceeding without mask.")
        return None

def apply_theta_gate(dtheta, dphi, gate_deg):
    alpha = np.arctan2(dtheta, dphi)
    return (np.abs(alpha) <= np.deg2rad(gate_deg)), alpha

def rotation_nulls(dtheta, dphi, angles_deg):
    xs, ys = [], []
    for ang in angles_deg:
        r = np.deg2rad(ang); c, s = np.cos(r), np.sin(r)
        xs.append(dphi*c - dtheta*s)
        ys.append(dphi*s + dtheta*c)
    return xs, ys

def binom_stats(k, n, p0=0.5):
    frac = k/n if n>0 else np.nan
    se = math.sqrt(frac*(1-frac)/n) if n>0 else np.nan
    ci_lo = max(0.0, frac - 1.96*se) if n>0 else np.nan
    ci_hi = min(1.0, frac + 1.96*se) if n>0 else np.nan
    try:
        from scipy.stats import binomtest
        pval = binomtest(k, n, p=p0, alternative="greater").pvalue
    except Exception:
        pval = np.nan
    z = (frac - p0)/math.sqrt(p0*(1-p0)/n) if n>0 else np.nan
    return dict(frac=frac, n=n, k=k, ci_lo=ci_lo, ci_hi=ci_hi, z=z, p_one_sided=pval)

def summarize_alignment(spins, dphi, dtheta, theta_gate_deg, sky_ok=None):
    gate_mask, _ = apply_theta_gate(dtheta, dphi, theta_gate_deg)
    pred = np.where(dphi >= 0, 1, -1)
    valid = gate_mask & np.isfinite(spins) & np.isfinite(dphi)
    if sky_ok is not None: valid &= sky_ok
    N = int(valid.sum())
    if N == 0: return dict(stats=None, valid_mask=valid, pred_spin=pred)
    k = int((spins[valid].astype(int) == pred[valid].astype(int)).sum())
    return dict(stats=binom_stats(k, N), valid_mask=valid, pred_spin=pred)

def run_spin_shuffle_nulls(spins, dphi, dtheta, theta_gate_deg, n_iter=200):
    gate_mask,_ = apply_theta_gate(dtheta, dphi, theta_gate_deg)
    valid = gate_mask & np.isfinite(spins) & np.isfinite(dphi)
    if valid.sum()==0: return np.array([])
    s = spins[valid].astype(int).copy()
    p = np.where(dphi[valid]>=0, 1, -1).astype(int)
    out=[]
    for _ in range(int(n_iter)):
        np.random.shuffle(s)
        out.append((s==p).mean())
    return np.array(out)

def write_zip(out_dir, zip_path):
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(out_dir):
            for fn in files:
                fp = os.path.join(root, fn)
                zf.write(fp, arcname=os.path.relpath(fp, out_dir))

# --------- Discover inputs ---------
dtheta_path = discover_map("dtheta")
dphi_path   = discover_map("dphi")
gz1_path    = discover_gz1()
mask_path   = discover_mask()

print("Auto-discovery:")
print("  dtheta →", dtheta_path)
print("  dphi   →", dphi_path)
print("  gz1    →", gz1_path)
print("  mask   →", mask_path if mask_path else "(none)")

if not dtheta_path or not dphi_path: raise FileNotFoundError("Missing gradient maps.")
if not gz1_path: raise FileNotFoundError("Could not find Galaxy Zoo file (table2).")

# --------- Load maps ---------
dtheta_map = np.load(dtheta_path)
dphi_map   = np.load(dphi_path)
nside = infer_nside(dtheta_map)
if infer_nside(dphi_map) != nside: raise ValueError("dtheta/dphi NSIDEs differ.")
mask_vec = try_load_mask(mask_path, nside)

# --------- Load Galaxy Zoo + sexagesimal-safe RA/Dec ---------
try:
    gz1 = pd.read_csv(gz1_path, compression="infer")
except Exception:
    gz1 = pd.read_csv(gz1_path)
ra_col, dec_col = detect_ra_dec_columns(gz1)
spin_obs, keep  = extract_spin_gz1(gz1)
gz1 = gz1.loc[keep].copy()
spin_obs = spin_obs.loc[gz1.index]

# robust RA/Dec parsing (deg or HMS/DMS)
ra_deg, dec_deg = parse_ra_dec_mixed(gz1[ra_col], gz1[dec_col])

# drop rows that failed to parse
ok = np.isfinite(ra_deg) & np.isfinite(dec_deg)
gz1 = gz1.loc[ok].copy()
spin_obs = spin_obs.loc[gz1.index]
ra_deg = ra_deg[ok]; dec_deg = dec_deg[ok]

thetas = (np.pi/2.0) - np.deg2rad(dec_deg)
phis   = np.deg2rad(ra_deg) % (2*np.pi)
pix = hp.ang2pix(nside, thetas, phis, nest=False)
dtheta = dtheta_map[pix]
dphi   = dphi_map[pix]
sky_ok = (mask_vec[pix] > 0) if mask_vec is not None else np.ones_like(dphi, dtype=bool)

# --------- Compute ---------
res = summarize_alignment(spin_obs.values, dphi, dtheta, THETA_GATE_DEG, sky_ok=sky_ok)
stats = res["stats"]; valid_mask = res["valid_mask"]; pred_spin = res["pred_spin"]

# Rotation nulls
rot_xs, rot_ys = rotation_nulls(dtheta[valid_mask], dphi[valid_mask], ROTATIONS_DEG)
rot_fracs = []
for y_rot in rot_ys:
    pred_rot = np.where(y_rot>=0, 1, -1)
    rot_fracs.append((spin_obs.values[valid_mask].astype(int) == pred_rot.astype(int)).mean())

# Shuffle nulls
N_valid = int(valid_mask.sum())
if N_valid > 250_000:
    idx = np.random.choice(np.where(valid_mask)[0], size=250_000, replace=False)
    shuffle_fracs = run_spin_shuffle_nulls(spin_obs.values[idx], dphi[idx], dtheta[idx], THETA_GATE_DEG, n_iter=MAX_SHUFFLE_N)
else:
    shuffle_fracs = run_spin_shuffle_nulls(spin_obs.values[valid_mask], dphi[valid_mask], dtheta[valid_mask], THETA_GATE_DEG, n_iter=MAX_SHUFFLE_N)

# --------- Save outputs ---------
stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_dir = f"/mnt/data/Mechanism1_OptionA_GZ1_{stamp}"
os.makedirs(out_dir, exist_ok=True)

summary = {
    "dataset": "GalaxyZoo1",
    "maps": {"dtheta": dtheta_path, "dphi": dphi_path, "mask": mask_path},
    "gz1_path": gz1_path,
    "nside": int(nside),
    "theta_gate_deg": THETA_GATE_DEG,
    "min_margin": MIN_MARGIN,
    "rotations_deg": ROTATIONS_DEG,
    "random_seed": RANDOM_SEED,
    "N_after_filters": int(N_valid),
    "alignment": stats,
    "rotation_nulls": dict(zip([str(x) for x in ROTATIONS_DEG], [float(x) for x in rot_fracs])),
    "shuffle_null_mean": float(np.mean(shuffle_fracs)) if shuffle_fracs.size else None,
    "shuffle_null_std": float(np.std(shuffle_fracs)) if shuffle_fracs.size else None,
    "shuffle_null_iters": int(shuffle_fracs.size),
}
pd.DataFrame({
    "valid": valid_mask.astype(int),
    "spin_obs": spin_obs.values.astype(int),
    "pred_spin": pred_spin.astype(int),
    "dphi": dphi.astype(float),
    "dtheta": dtheta.astype(float),
}).to_csv(os.path.join(out_dir, "gz1_per_object_vectors.csv"), index=False)
pd.DataFrame({"rotation_deg": ROTATIONS_DEG, "rot_frac": rot_fracs}).to_csv(
    os.path.join(out_dir, "rotation_nulls.csv"), index=False)
if shuffle_fracs.size:
    pd.DataFrame({"shuffle_frac": shuffle_fracs}).to_csv(
        os.path.join(out_dir, "shuffle_nulls.csv"), index=False)
with open(os.path.join(out_dir, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
with open(os.path.join(out_dir, "alignment_summary.txt"), "w") as f:
    s = summary["alignment"]
    f.write(
        "Mechanism 1 · Option A — Galaxy Zoo (sexagesimal-safe)\n"
        f"GZ1 file: {gz1_path}\n"
        f"Maps: dtheta={dtheta_path}\n      dphi  ={dphi_path}\n"
        f"NSIDE={summary['nside']}\n"
        f"θ-gate = ±{THETA_GATE_DEG}° ; MIN_MARGIN={MIN_MARGIN}\n"
        f"N (valid) = {summary['N_after_filters']}\n"
        f"Alignment fraction = {s['frac']:.6f}  (95% CI [{s['ci_lo']:.6f}, {s['ci_hi']:.6f}])\n"
        f"z vs 0.5 = {s['z']:.3f} ; one-sided p = {s['p_one_sided']:.3e}\n"
        f"Rotation nulls (deg→frac): {summary['rotation_nulls']}\n"
        f"Shuffle null mean±std = {summary['shuffle_null_mean']:.6f} ± {summary['shuffle_null_std']:.6f} "
        f"(iters={summary['shuffle_null_iters']})\n"
    )
plt.figure(figsize=(6,4))
plt.title("Galaxy Zoo — Option A alignment")
plt.axhline(0.5, linestyle="--")
plt.bar(["Observed"], [summary["alignment"]["frac"]])
plt.ylabel("Alignment fraction"); plt.ylim(0.45, 0.75); plt.tight_layout()
plt.savefig(os.path.join(out_dir, "plot_alignment.png"), dpi=160); plt.close()

zip_path = f"{out_dir}.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(out_dir):
        for fn in files:
            fp = os.path.join(root, fn)
            zf.write(fp, arcname=os.path.relpath(fp, out_dir))

print("\nDONE.")
print(f"Results folder: {out_dir}")
print(f"ZIP bundle:     {zip_path}")


Auto-discovery:
  dtheta → ./Logosfield_dtheta_map.npy
  dphi   → /content/Logosfield_dphi_map.npy
  gz1    → ./GalaxyZoo1_DR_table2.csv.gz
  mask   → ./glimpse_mask.fits


/tmp/ipython-input-1678358610.py:266: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")



DONE.
Results folder: /mnt/data/Mechanism1_OptionA_GZ1_20250831_010057
ZIP bundle:     /mnt/data/Mechanism1_OptionA_GZ1_20250831_010057.zip


In [36]:
# ==== Mechanism 1 · Option A (Galaxy Zoo only) — AUTO-DISCOVER + SEXAGESIMAL-SAFE + AUTO-DOWNLOAD ====
import os, sys, json, math, zipfile, random, glob
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt

THETA_GATE_DEG = 15.0
MIN_MARGIN = 0.05
ROTATIONS_DEG = [0,30,60,90]
MAX_SHUFFLE_N = 200
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED)

# --------- Deps ---------
def ensure_healpy_astropy():
    try:
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "healpy", "astropy"])
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
hp, SkyCoord, u = ensure_healpy_astropy()

# --------- Helpers ---------
def infer_nside(vec):
    n = int(vec.size); ns = int(round((n/12.0)**0.5))
    if 12*(ns**2) != n: raise ValueError(f"Map length {n} not 12*nside^2.")
    return ns

def autodiscover(patterns, roots=("/mnt/data","/content",".")):
    cands=[]
    for r in roots:
        if not os.path.isdir(r): continue
        for pat in patterns:
            cands += glob.glob(os.path.join(r, pat))
    # Prefer names with 'galaxyzoo' / shorter names
    cands = sorted(set(cands), key=lambda p: (0 if "galaxyzoo" in p.lower() else 1, len(os.path.basename(p))))
    return cands[0] if cands else None

def discover_gz1():
    return autodiscover([
        "GalaxyZoo*table2*.csv*","GalaxyZoo*DR*table2*.csv*","*GZ*table2*.csv*","*galaxy*zoo*table2*.csv*"
    ])

def discover_map(kind):  # 'dtheta'|'dphi'
    return autodiscover([f"*{kind}*.npy", f"*Logosfield*{kind}*.npy", f"*{kind}_map*.npy", f"*{kind} map*.npy"])

def discover_mask():
    return autodiscover(["*mask*.fits"])

def detect_ra_dec_columns(df):
    L = {c.lower(): c for c in df.columns}
    ra = next((L[k] for k in ("ra","ra_deg","ra (deg)","ra_deg_j2000","ra_j2000") if k in L),
              next((c for c in df.columns if "ra" in c.lower()), None))
    dec = next((L[k] for k in ("dec","dec_deg","dec (deg)","dec_deg_j2000","dec_j2000","de") if k in L),
               next((c for c in df.columns if "dec" in c.lower() or c.lower()=="de"), None))
    if ra is None or dec is None: raise ValueError("RA/Dec columns not found.")
    return ra, dec

def parse_ra_dec_mixed(ra_series, dec_series):
    """Return (ra_deg, dec_deg) handling degrees or HMS/DMS strings."""
    # Quick numeric path
    ra_num  = pd.to_numeric(ra_series,  errors="coerce")
    dec_num = pd.to_numeric(dec_series, errors="coerce")
    if ra_num.notna().all() and dec_num.notna().all():
        return ra_num.values.astype(float), dec_num.values.astype(float)

    # Sexagesimal or mixed -> use SkyCoord
    ra_str  = ra_series.astype(str).str.strip()
    dec_str = dec_series.astype(str).str.strip()
    if (ra_str.str.contains(":").mean() > 0.05) or (dec_str.str.contains(":").mean() > 0.05):
        sc = SkyCoord(ra=ra_str.values, dec=dec_str.values, unit=(u.hourangle, u.deg), frame="icrs")
        return sc.ra.deg.astype(float), sc.dec.deg.astype(float)

    # Last fallback: attempt per-row deg strings
    out_ra, out_dec = [], []
    for r, d in zip(ra_series, dec_series):
        try:
            out_ra.append(float(r)); out_dec.append(float(d))
        except Exception:
            try:
                sc = SkyCoord(str(r), str(d), unit=(u.deg, u.deg), frame="icrs")
                out_ra.append(float(sc.ra.deg)); out_dec.append(float(sc.dec.deg))
            except Exception:
                out_ra.append(np.nan); out_dec.append(np.nan)
    return np.array(out_ra), np.array(out_dec)

def extract_spin_gz1(df):
    p_cw = next((c for c in df.columns if c.lower() in ("p_cw","p(cw)","p_cw_prob","prob_cw")), None)
    p_acw = next((c for c in df.columns if c.lower() in ("p_acw","p(ccw)","p_acw_prob","prob_acw","p_ccw","prob_ccw")), None)
    if p_cw and p_acw:
        pc = pd.to_numeric(df[p_cw], errors="coerce").astype(float)
        pa = pd.to_numeric(df[p_acw], errors="coerce").astype(float)
        keep = (pc - pa).abs() >= MIN_MARGIN
        spin = np.where(pc > pa, 1, -1).astype(int)
        return pd.Series(spin, index=df.index), keep
    for c in df.columns:
        if c.lower() in ("spin","handedness","spiral","cw_ccw"):
            vals = df[c].astype(str).str.lower().str.strip()
            spin = np.where(vals.isin(["cw","+1","1","clockwise"]), 1,
                            np.where(vals.isin(["ccw","-1","counterclockwise","anticlockwise"]), -1, np.nan))
            keep = ~np.isnan(spin)
            return pd.Series(spin, index=df.index).astype(int), keep
    cw_flag = next((c for c in df.columns if "cw" in c.lower() and "flag" in c.lower()), None)
    ccw_flag = next((c for c in df.columns if "ccw" in c.lower() and "flag" in c.lower()), None)
    if cw_flag and ccw_flag:
        spin = np.where(df[cw_flag].astype(int)==1, 1,
                        np.where(df[ccw_flag].astype(int)==1, -1, np.nan))
        keep = ~np.isnan(spin)
        return pd.Series(spin, index=df.index).astype(int), keep
    raise ValueError("Could not find spins (P_CW/P_ACW or spin labels).")

def try_load_mask(mask_path, nside_maps):
    try:
        from astropy.io import fits
    except Exception:
        print("astropy not present; continuing without mask.")
        return None
    if not mask_path or not os.path.exists(mask_path): return None
    try:
        with fits.open(mask_path) as hdul:
            data = hdul[1].data if len(hdul)>1 else hdul[0].data
            vec = np.array(data).astype(float).ravel()
        ns = infer_nside(vec)
        return vec if ns==nside_maps else hp.ud_grade(vec, nside_maps, power=-2)
    except Exception as e:
        print(f"Mask load/resample failed: {e}; proceeding without mask.")
        return None

def apply_theta_gate(dtheta, dphi, gate_deg):
    alpha = np.arctan2(dtheta, dphi)
    return (np.abs(alpha) <= np.deg2rad(gate_deg)), alpha

def rotation_nulls(dtheta, dphi, angles_deg):
    xs, ys = [], []
    for ang in angles_deg:
        r = np.deg2rad(ang); c, s = np.cos(r), np.sin(r)
        xs.append(dphi*c - dtheta*s)
        ys.append(dphi*s + dtheta*c)
    return xs, ys

def binom_stats(k, n, p0=0.5):
    frac = k/n if n>0 else np.nan
    se = math.sqrt(frac*(1-frac)/n) if n>0 else np.nan
    ci_lo = max(0.0, frac - 1.96*se) if n>0 else np.nan
    ci_hi = min(1.0, frac + 1.96*se) if n>0 else np.nan
    try:
        from scipy.stats import binomtest
        pval = binomtest(k, n, p=p0, alternative="greater").pvalue
    except Exception:
        pval = np.nan
    z = (frac - p0)/math.sqrt(p0*(1-p0)/n) if n>0 else np.nan
    return dict(frac=frac, n=n, k=k, ci_lo=ci_lo, ci_hi=ci_hi, z=z, p_one_sided=pval)

def summarize_alignment(spins, dphi, dtheta, theta_gate_deg, sky_ok=None):
    gate_mask, _ = apply_theta_gate(dtheta, dphi, theta_gate_deg)
    pred = np.where(dphi >= 0, 1, -1)
    valid = gate_mask & np.isfinite(spins) & np.isfinite(dphi)
    if sky_ok is not None: valid &= sky_ok
    N = int(valid.sum())
    if N == 0: return dict(stats=None, valid_mask=valid, pred_spin=pred)
    k = int((spins[valid].astype(int) == pred[valid].astype(int)).sum())
    return dict(stats=binom_stats(k, N), valid_mask=valid, pred_spin=pred)

def run_spin_shuffle_nulls(spins, dphi, dtheta, theta_gate_deg, n_iter=200):
    gate_mask,_ = apply_theta_gate(dtheta, dphi, theta_gate_deg)
    valid = gate_mask & np.isfinite(spins) & np.isfinite(dphi)
    if valid.sum()==0: return np.array([])
    s = spins[valid].astype(int).copy()
    p = np.where(dphi[valid]>=0, 1, -1).astype(int)
    out=[]
    for _ in range(int(n_iter)):
        np.random.shuffle(s)
        out.append((s==p).mean())
    return np.array(out)

def write_zip(out_dir, zip_path):
    with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(out_dir):
            for fn in files:
                fp = os.path.join(root, fn)
                zf.write(fp, arcname=os.path.relpath(fp, out_dir))

# --------- Discover inputs ---------
dtheta_path = discover_map("dtheta")
dphi_path   = discover_map("dphi")
gz1_path    = discover_gz1()
mask_path   = discover_mask()

print("Auto-discovery:")
print("  dtheta →", dtheta_path)
print("  dphi   →", dphi_path)
print("  gz1    →", gz1_path)
print("  mask   →", mask_path if mask_path else "(none)")

if not dtheta_path or not dphi_path: raise FileNotFoundError("Missing gradient maps.")
if not gz1_path: raise FileNotFoundError("Could not find Galaxy Zoo file (table2).")

# --------- Load maps ---------
dtheta_map = np.load(dtheta_path)
dphi_map   = np.load(dphi_path)
nside = infer_nside(dtheta_map)
if infer_nside(dphi_map) != nside: raise ValueError("dtheta/dphi NSIDEs differ.")
mask_vec = try_load_mask(mask_path, nside)

# --------- Load Galaxy Zoo (sexagesimal-safe RA/Dec) ---------
try:
    gz1 = pd.read_csv(gz1_path, compression="infer")
except Exception:
    gz1 = pd.read_csv(gz1_path)
ra_col, dec_col = detect_ra_dec_columns(gz1)
spin_obs, keep  = extract_spin_gz1(gz1)
gz1 = gz1.loc[keep].copy()
spin_obs = spin_obs.loc[gz1.index]

ra_deg, dec_deg = parse_ra_dec_mixed(gz1[ra_col], gz1[dec_col])
ok = np.isfinite(ra_deg) & np.isfinite(dec_deg)
gz1 = gz1.loc[ok].copy(); spin_obs = spin_obs.loc[gz1.index]
ra_deg = ra_deg[ok]; dec_deg = dec_deg[ok]

thetas = (np.pi/2.0) - np.deg2rad(dec_deg)
phis   = np.deg2rad(ra_deg) % (2*np.pi)
pix = hp.ang2pix(nside, thetas, phis, nest=False)
dtheta = dtheta_map[pix]
dphi   = dphi_map[pix]
sky_ok = (mask_vec[pix] > 0) if mask_vec is not None else np.ones_like(dphi, dtype=bool)

# --------- Compute alignment + nulls ---------
res = summarize_alignment(spin_obs.values, dphi, dtheta, THETA_GATE_DEG, sky_ok=sky_ok)
stats = res["stats"]; valid_mask = res["valid_mask"]; pred_spin = res["pred_spin"]

rot_xs, rot_ys = rotation_nulls(dtheta[valid_mask], dphi[valid_mask], ROTATIONS_DEG)
rot_fracs = []
for y_rot in rot_ys:
    pred_rot = np.where(y_rot>=0, 1, -1)
    rot_fracs.append((spin_obs.values[valid_mask].astype(int) == pred_rot.astype(int)).mean())

N_valid = int(valid_mask.sum())
if N_valid > 250_000:
    idx = np.random.choice(np.where(valid_mask)[0], size=250_000, replace=False)
    shuffle_fracs = run_spin_shuffle_nulls(spin_obs.values[idx], dphi[idx], dtheta[idx], THETA_GATE_DEG, n_iter=MAX_SHUFFLE_N)
else:
    shuffle_fracs = run_spin_shuffle_nulls(spin_obs.values[valid_mask], dphi[valid_mask], dtheta[valid_mask], THETA_GATE_DEG, n_iter=MAX_SHUFFLE_N)

# --------- Save outputs ---------
stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_dir = f"/mnt/data/Mechanism1_OptionA_GZ1_{stamp}"
os.makedirs(out_dir, exist_ok=True)

summary = {
    "dataset": "GalaxyZoo1",
    "maps": {"dtheta": dtheta_path, "dphi": dphi_path, "mask": mask_path},
    "gz1_path": gz1_path,
    "nside": int(nside),
    "theta_gate_deg": THETA_GATE_DEG,
    "min_margin": MIN_MARGIN,
    "rotations_deg": ROTATIONS_DEG,
    "random_seed": RANDOM_SEED,
    "N_after_filters": int(N_valid),
    "alignment": stats,
    "rotation_nulls": dict(zip([str(x) for x in ROTATIONS_DEG], [float(x) for x in rot_fracs])),
    "shuffle_null_mean": float(np.mean(shuffle_fracs)) if shuffle_fracs.size else None,
    "shuffle_null_std": float(np.std(shuffle_fracs)) if shuffle_fracs.size else None,
    "shuffle_null_iters": int(shuffle_fracs.size),
}
pd.DataFrame({
    "valid": valid_mask.astype(int),
    "spin_obs": spin_obs.values.astype(int),
    "pred_spin": pred_spin.astype(int),
    "dphi": dphi.astype(float),
    "dtheta": dtheta.astype(float),
}).to_csv(os.path.join(out_dir, "gz1_per_object_vectors.csv"), index=False)
pd.DataFrame({"rotation_deg": ROTATIONS_DEG, "rot_frac": rot_fracs}).to_csv(
    os.path.join(out_dir, "rotation_nulls.csv"), index=False)
if shuffle_fracs.size:
    pd.DataFrame({"shuffle_frac": shuffle_fracs}).to_csv(
        os.path.join(out_dir, "shuffle_nulls.csv"), index=False)
with open(os.path.join(out_dir, "summary.json"), "w") as f:
    json.dump(summary, f, indent=2)
with open(os.path.join(out_dir, "alignment_summary.txt"), "w") as f:
    s = summary["alignment"]
    f.write(
        "Mechanism 1 · Option A — Galaxy Zoo (sexagesimal-safe)\n"
        f"GZ1 file: {gz1_path}\n"
        f"Maps: dtheta={dtheta_path}\n      dphi  ={dphi_path}\n"
        f"NSIDE={summary['nside']}\n"
        f"θ-gate = ±{THETA_GATE_DEG}° ; MIN_MARGIN={MIN_MARGIN}\n"
        f"N (valid) = {summary['N_after_filters']}\n"
        f"Alignment fraction = {s['frac']:.6f}  (95% CI [{s['ci_lo']:.6f}, {s['ci_hi']:.6f}])\n"
        f"z vs 0.5 = {s['z']:.3f} ; one-sided p = {s['p_one_sided']:.3e}\n"
        f"Rotation nulls (deg→frac): {summary['rotation_nulls']}\n"
        f"Shuffle null mean±std = {summary['shuffle_null_mean']:.6f} ± {summary['shuffle_null_std']:.6f} "
        f"(iters={summary['shuffle_null_iters']})\n"
    )
plt.figure(figsize=(6,4))
plt.title("Galaxy Zoo — Option A alignment")
plt.axhline(0.5, linestyle="--")
plt.bar(["Observed"], [summary["alignment"]["frac"]])
plt.ylabel("Alignment fraction"); plt.ylim(0.45, 0.75); plt.tight_layout()
plt.savefig(os.path.join(out_dir, "plot_alignment.png"), dpi=160); plt.close()

# --------- ZIP + auto-download ---------
zip_path = f"{out_dir}.zip"
write_zip(out_dir, zip_path)

print("\nDONE.")
print(f"Results folder: {out_dir}")
print(f"ZIP bundle:     {zip_path}")

# Trigger downloadable file in Colab (safe no-op elsewhere)
try:
    from google.colab import files
    files.download(zip_path)
except Exception as e:
    print("Colab download not available in this environment:", e)


Auto-discovery:
  dtheta → ./Logosfield_dtheta_map.npy
  dphi   → /content/Logosfield_dphi_map.npy
  gz1    → ./GalaxyZoo1_DR_table2.csv.gz
  mask   → ./glimpse_mask.fits


/tmp/ipython-input-237797779.py:251: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")



DONE.
Results folder: /mnt/data/Mechanism1_OptionA_GZ1_20250831_010805
ZIP bundle:     /mnt/data/Mechanism1_OptionA_GZ1_20250831_010805.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [37]:
# === Option A calibration sweep — find the convention that matches validated results ===
import os, sys, json, math, glob, zipfile, random
from datetime import datetime
import numpy as np, pandas as pd

# ---- discover the same inputs as before ----
def autodiscover(pats, roots=("/mnt/data","/content",".")):
    out=[]
    for r in roots:
        if not os.path.isdir(r): continue
        for p in pats: out += glob.glob(os.path.join(r,p))
    return sorted(set(out), key=lambda p:(len(os.path.basename(p)), p.lower()))

def discover_map(kind):  # 'dtheta'|'dphi'
    for patset in [[f"*{kind}*.npy"], [f"*Logosfield*{kind}*.npy"], [f"*{kind}_map*.npy"], [f"*{kind} map*.npy"]]:
        c=autodiscover(patset)
        if c: return c[0]
    return None

def discover_gz1():
    c = autodiscover(["GalaxyZoo*table2*.csv*","*GZ*table2*.csv*","*galaxy*zoo*table2*.csv*"])
    return c[0] if c else None

def discover_mask():
    c = autodiscover(["*mask*.fits"])
    return c[0] if c else None

# ---- deps ----
def ensure_healpy_astropy():
    try:
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable,"-m","pip","install","-q","healpy","astropy"])
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        return hp, SkyCoord, u
hp, SkyCoord, u = ensure_healpy_astropy()

# ---- basic helpers ----
def infer_nside(vec):
    n = int(vec.size); ns = int(round((n/12.0)**0.5))
    if 12*(ns**2) != n: raise ValueError("Map length doesn't match 12*nside^2.")
    return ns

def detect_ra_dec_columns(df):
    L={c.lower():c for c in df.columns}
    ra  = L.get("ra") or L.get("ra_deg") or L.get("ra (deg)") or next((c for c in df.columns if "ra" in c.lower()),None)
    dec = L.get("dec") or L.get("dec_deg") or L.get("dec (deg)") or L.get("de") or next((c for c in df.columns if "dec" in c.lower() or c.lower()=="de"),None)
    if ra is None or dec is None: raise ValueError("RA/Dec not found.")
    return ra, dec

def parse_ra_dec_mixed(ra_series, dec_series):
    ra_num  = pd.to_numeric(ra_series, errors="coerce")
    dec_num = pd.to_numeric(dec_series, errors="coerce")
    if ra_num.notna().all() and dec_num.notna().all():
        return ra_num.values.astype(float), dec_num.values.astype(float)
    ra_str = ra_series.astype(str).str.strip()
    dec_str= dec_series.astype(str).str.strip()
    sc = SkyCoord(ra=ra_str.values, dec=dec_str.values, unit=(u.hourangle, u.deg), frame="icrs")
    return sc.ra.deg.astype(float), sc.dec.deg.astype(float)

def extract_spin_gz1(df, min_margin=0.05):
    p_cw = next((c for c in df.columns if c.lower() in ("p_cw","p(cw)","p_cw_prob","prob_cw")), None)
    p_acw= next((c for c in df.columns if c.lower() in ("p_acw","p(ccw)","p_acw_prob","prob_acw","p_ccw","prob_ccw")), None)
    if p_cw and p_acw:
        pc = pd.to_numeric(df[p_cw], errors="coerce").astype(float)
        pa = pd.to_numeric(df[p_acw], errors="coerce").astype(float)
        keep = (pc - pa).abs() >= min_margin
        spin = np.where(pc > pa, 1, -1).astype(int)
        return pd.Series(spin, index=df.index), keep
    # fallback labels
    for c in df.columns:
        if c.lower() in ("spin","handedness","spiral","cw_ccw"):
            vals=df[c].astype(str).str.lower().str.strip()
            spin=np.where(vals.isin(["cw","+1","1","clockwise"]),1,
                          np.where(vals.isin(["ccw","-1","counterclockwise","anticlockwise"]),-1,np.nan))
            keep=~np.isnan(spin); return pd.Series(spin,index=df.index).astype(int), keep
    raise ValueError("No spin columns found.")

def binom_stats(k,n):
    frac=k/n if n else np.nan
    se=(frac*(1-frac)/n)**0.5 if n else np.nan
    return frac, max(0.0, frac-1.96*se) if n else np.nan, min(1.0, frac+1.96*se) if n else np.nan

def apply_gate(dth,dph,gate_deg):
    alpha=np.arctan2(dth,dph)
    return np.abs(alpha) <= np.deg2rad(gate_deg)

# ---- load data ----
dtheta_path = discover_map("dtheta"); dphi_path = discover_map("dphi")
gz1_path = discover_gz1(); mask_path = discover_mask()
print("Using:\n  dtheta:", dtheta_path, "\n  dphi:  ", dphi_path, "\n  gz1:   ", gz1_path, "\n  mask:  ", mask_path or "(none)")

dtheta_map = np.load(dtheta_path); dphi_map = np.load(dphi_path)
nside = infer_nside(dtheta_map)
assert infer_nside(dphi_map)==nside, "NSIDE mismatch."

# mask (optional)
mask_vec=None
if mask_path:
    try:
        from astropy.io import fits
        with fits.open(mask_path) as hdul:
            data = hdul[1].data if len(hdul)>1 else hdul[0].data
            vec = np.array(data).astype(float).ravel()
        ns_mask = infer_nside(vec)
        mask_vec = vec if ns_mask==nside else hp.ud_grade(vec, nside, power=-2)
    except Exception as e:
        print("Mask not used:", e)

# galaxy zoo
try:
    gz1 = pd.read_csv(gz1_path, compression="infer")
except Exception:
    gz1 = pd.read_csv(gz1_path)
ra_col, dec_col = detect_ra_dec_columns(gz1)
spin_obs, keep = extract_spin_gz1(gz1, min_margin=0.05)
gz1 = gz1.loc[keep].copy(); spin_obs = spin_obs.loc[gz1.index]
ra_deg, dec_deg = parse_ra_dec_mixed(gz1[ra_col], gz1[dec_col])
ok = np.isfinite(ra_deg) & np.isfinite(dec_deg)
gz1 = gz1.loc[ok].copy(); spin_obs = spin_obs.loc[gz1.index]
ra_deg = ra_deg[ok]; dec_deg = dec_deg[ok]

# precompute angles and two pix modes
thetas = (np.pi/2.0) - np.deg2rad(dec_deg)
phis   = np.deg2rad(ra_deg) % (2*np.pi)
pix_ring = hp.ang2pix(nside, thetas, phis, nest=False)
pix_nest = hp.ang2pix(nside, thetas, phis, nest=True)

# ---- sweep space ----
gate_list = [10,15,20]
nest_opts = [False, True]
swap_opts = [False, True]      # swap dtheta<->dphi
flip_dth  = [0,1]              # 1 means multiply by -1
flip_dph  = [0,1]
phi_metric = ["none","times_sin","div_sin"]  # component tweak
pred_sign = ["+dphi","-dphi"]  # handedness flip
use_mask  = [True, False]

records=[]

for gate in gate_list:
    for nest in nest_opts:
        pix = pix_nest if nest else pix_ring
        raw_dth = dtheta_map[pix].astype(float)
        raw_dph = dphi_map[pix].astype(float)

        for swap in swap_opts:
            dth = raw_dth.copy(); dph = raw_dph.copy()
            if swap: dth, dph = dph, dth

            for fth in flip_dth:
                dth_s = -dth if fth else dth
                for fph in flip_dph:
                    dph_s = -dph if fph else dph

                    # metric tweak
                    for met in phi_metric:
                        if met=="times_sin":
                            dph_m = dph_s * np.sin(thetas)
                        elif met=="div_sin":
                            # avoid poles
                            s = np.sin(thetas)
                            s[s==0] = 1.0
                            dph_m = dph_s / s
                        else:
                            dph_m = dph_s

                        for ps in pred_sign:
                            pred = np.where(dph_m>=0, 1, -1)
                            if ps=="-dphi": pred = -pred

                            for mflag in use_mask:
                                gate_mask = apply_gate(dth_s, dph_m, gate)
                                valid = gate_mask & np.isfinite(dph_m) & np.isfinite(dth_s)
                                if mflag and (mask_vec is not None):
                                    valid = valid & (mask_vec[(pix_nest if nest else pix_ring)] > 0)

                                spins = spin_obs.values
                                N = int(valid.sum())
                                if N==0: continue
                                aligned = (spins[valid].astype(int) == pred[valid].astype(int))
                                k = int(aligned.sum())
                                frac, lo, hi = binom_stats(k,N)
                                records.append(dict(
                                    frac=frac, ci_lo=lo, ci_hi=hi, N=N, k=k,
                                    gate=gate, nest=nest, swap=swap, flip_dtheta=fth, flip_dphi=fph,
                                    phi_metric=met, pred_sign=ps, mask=mflag
                                ))

res = pd.DataFrame.from_records(records).sort_values(["frac","N"], ascending=[False,False]).reset_index(drop=True)
print("Top 10 configurations:")
display(res.head(10))

# Save & auto-download
stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_csv = f"/mnt/data/OptionA_calibration_sweep_{stamp}.csv"
res.to_csv(out_csv, index=False)
print("Saved sweep table:", out_csv)

try:
    from google.colab import files
    files.download(out_csv)
except Exception as e:
    print("Colab download not available:", e)


Using:
  dtheta: ./Logosfield_dtheta_map.npy 
  dphi:   ./Logosfield_dphi_map.npy 
  gz1:    ./GalaxyZoo1_DR_table2.csv.gz 
  mask:   ./glimpse_mask.fits
Top 10 configurations:


,frac,ci_lo,ci_hi,N,k,gate,nest,swap,flip_dtheta,flip_dphi,phi_metric,pred_sign,mask
0,0.571429,0.204823,0.938035,7,4,20,True,True,0,0,none,+dphi,True
1,0.571429,0.204823,0.938035,7,4,20,True,True,0,0,times_sin,+dphi,True
2,0.571429,0.204823,0.938035,7,4,20,True,True,0,0,div_sin,+dphi,True
3,0.571429,0.204823,0.938035,7,4,20,True,True,1,0,none,+dphi,True
4,0.571429,0.204823,0.938035,7,4,20,True,True,1,0,times_sin,+dphi,True
5,0.571429,0.204823,0.938035,7,4,20,True,True,1,0,div_sin,+dphi,True
6,0.557775,0.536548,0.579002,2103,1173,20,True,True,0,0,div_sin,-dphi,False
7,0.557775,0.536548,0.579002,2103,1173,20,True,True,1,0,div_sin,-dphi,False
8,0.556976,0.534509,0.579442,1878,1046,20,True,True,0,0,none,-dphi,False
9,0.556976,0.534509,0.579442,1878,1046,20,True,True,1,0,none,-dphi,False


Saved sweep table: /mnt/data/OptionA_calibration_sweep_20250831_011408.csv


/tmp/ipython-input-12392127.py:201: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
# === Mechanism 1 · Option A (Galaxy Zoo) — CORRECTED SETTINGS + optional mask, auto-download ===
import os, sys, json, math, zipfile, random, glob
from datetime import datetime
import numpy as np, pandas as pd, matplotlib.pyplot as plt

# --- Config ---
THETA_GATE_DEG = 20.0          # from sweep
MIN_MARGIN = 0.05
MAX_SHUFFLE_N = 200
ROTATIONS_DEG = [0, 30, 60, 90]
EVALUATE_BOTH_MASK_SETTINGS = True   # set False to run only USE_MASK below
USE_MASK = False                     # used only if EVALUATE_BOTH_MASK_SETTINGS=False
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED)

# --- Deps ---
def ensure_healpy_astropy():
    try:
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        from astropy.io import fits
        return hp, SkyCoord, u, fits
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "healpy", "astropy"])
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        from astropy.io import fits
        return hp, SkyCoord, u, fits
hp, SkyCoord, u, fits = ensure_healpy_astropy()

# --- Helpers ---
def infer_nside(vec):
    n = int(vec.size); ns = int(round((n/12.0)**0.5))
    if 12*(ns**2) != n: raise ValueError(f"Map length {n} not 12*nside^2.")
    return ns

def autodiscover(pats, roots=("/mnt/data","/content",".")):
    out=[]
    for r in roots:
        if not os.path.isdir(r): continue
        for p in pats: out += glob.glob(os.path.join(r,p))
    return sorted(set(out), key=lambda p: (len(os.path.basename(p)), p.lower()))

def discover_map(kind):   # 'dtheta' | 'dphi'
    for pats in [[f"*{kind}*.npy"], [f"*Logosfield*{kind}*.npy"], [f"*{kind}_map*.npy"], [f"*{kind} map*.npy"]]:
        c = autodiscover(pats)
        if c: return c[0]
    return None

def discover_gz1():
    c = autodiscover(["GalaxyZoo*table2*.csv*","*GZ*table2*.csv*","*galaxy*zoo*table2*.csv*"])
    return c[0] if c else None

def discover_mask():
    c = autodiscover(["*mask*.fits"])
    return c[0] if c else None

def detect_ra_dec_columns(df):
    L = {c.lower(): c for c in df.columns}
    ra  = L.get("ra") or L.get("ra_deg") or L.get("ra (deg)") or next((c for c in df.columns if "ra" in c.lower()), None)
    dec = L.get("dec") or L.get("dec_deg") or L.get("dec (deg)") or L.get("de") or next((c for c in df.columns if "dec" in c.lower() or c.lower()=="de"), None)
    if ra is None or dec is None: raise ValueError("RA/Dec columns not found.")
    return ra, dec

def parse_ra_dec_mixed(ra_s, dec_s):
    ra_num  = pd.to_numeric(ra_s,  errors="coerce")
    dec_num = pd.to_numeric(dec_s, errors="coerce")
    if ra_num.notna().all() and dec_num.notna().all():
        return ra_num.values.astype(float), dec_num.values.astype(float)
    sc = SkyCoord(ra=ra_s.astype(str).values, dec=dec_s.astype(str).values, unit=(u.hourangle, u.deg), frame="icrs")
    return sc.ra.deg.astype(float), sc.dec.deg.astype(float)

def extract_spin_gz1(df, min_margin=0.05):
    p_cw = next((c for c in df.columns if c.lower() in ("p_cw","p(cw)","p_cw_prob","prob_cw")), None)
    p_acw= next((c for c in df.columns if c.lower() in ("p_acw","p(ccw)","p_acw_prob","prob_acw","p_ccw","prob_ccw")), None)
    if p_cw and p_acw:
        pc = pd.to_numeric(df[p_cw], errors="coerce").astype(float)
        pa = pd.to_numeric(df[p_acw], errors="coerce").astype(float)
        keep = (pc - pa).abs() >= min_margin
        spin = np.where(pc > pa, 1, -1).astype(int)
        return pd.Series(spin, index=df.index), keep
    for c in df.columns:
        if c.lower() in ("spin","handedness","spiral","cw_ccw"):
            vals = df[c].astype(str).str.lower().str.strip()
            spin = np.where(vals.isin(["cw","+1","1","clockwise"]), 1,
                            np.where(vals.isin(["ccw","-1","counterclockwise","anticlockwise"]), -1, np.nan))
            keep = ~np.isnan(spin)
            return pd.Series(spin, index=df.index).astype(int), keep
    raise ValueError("No spin columns found.")

def apply_theta_gate(dth, dph, gate_deg):
    alpha = np.arctan2(dth, dph)
    return np.abs(alpha) <= np.deg2rad(gate_deg), alpha

def rotation_nulls(dth, dph, angles_deg):
    xs, ys = [], []
    for ang in angles_deg:
        r = np.deg2rad(ang); c, s = np.cos(r), np.sin(r)
        xs.append(dph*c - dth*s)
        ys.append(dph*s + dth*c)
    return xs, ys

def binom_stats(k, n):
    frac = k/n if n else np.nan
    se = (frac*(1-frac)/n)**0.5 if n else np.nan
    lo = max(0.0, frac - 1.96*se) if n else np.nan
    hi = min(1.0, frac + 1.96*se) if n else np.nan
    return frac, lo, hi

def summarize(spins, dph_metric, dth_swap, gate_deg, sky_ok=None):
    # predict with handedness flip: -sign(dphi_metric)
    pred = -np.where(dph_metric >= 0, 1, -1)
    gate_mask, alpha = apply_theta_gate(dth_swap, dph_metric, gate_deg)
    valid = gate_mask & np.isfinite(dph_metric) & np.isfinite(dth_swap)
    if sky_ok is not None: valid &= sky_ok
    N = int(valid.sum())
    if N == 0:
        return dict(N=0)
    aligned = (spins[valid].astype(int) == pred[valid].astype(int))
    k = int(aligned.sum())
    frac, lo, hi = binom_stats(k, N)
    return dict(N=N, k=k, frac=frac, ci_lo=lo, ci_hi=hi, pred=pred, valid=valid, alpha=alpha)

def write_bundle(tag, valid, spins, pred, dph_m, dth_s, out_dir):
    import matplotlib.pyplot as plt
    # per-object CSV
    df = pd.DataFrame({
        "valid": valid.astype(int),
        "spin_obs": spins.astype(int),
        "pred_spin": pred.astype(int),
        "dphi_metric": dph_m.astype(float),
        "dtheta_swapped": dth_s.astype(float),
    })
    df.to_csv(os.path.join(out_dir, f"gz1_objects_{tag}.csv"), index=False)
    # simple plot
    plt.figure(figsize=(6,4))
    plt.title(f"Galaxy Zoo — Option A ({tag})")
    plt.axhline(0.5, ls="--")
    plt.bar(["Observed"], [ (df.loc[valid, "spin_obs"] == df.loc[valid, "pred_spin"]).mean() ])
    plt.ylabel("Alignment fraction"); plt.ylim(0.45, 0.75); plt.tight_layout()
    plt.savefig(os.path.join(out_dir, f"plot_{tag}.png"), dpi=160); plt.close()

# --- Discover inputs ---
dtheta_path = discover_map("dtheta")
dphi_path   = discover_map("dphi")
gz1_path    = discover_gz1()
mask_path   = discover_mask()

print("Inputs:")
print("  dtheta:", dtheta_path)
print("  dphi  :", dphi_path)
print("  gz1   :", gz1_path)
print("  mask  :", mask_path or "(none)")

# --- Load maps ---
dtheta_map = np.load(dtheta_path)
dphi_map   = np.load(dphi_path)
nside = infer_nside(dtheta_map)
assert infer_nside(dphi_map) == nside, "NSIDE mismatch."

# --- Load mask (optional) ---
mask_vec = None
if mask_path:
    try:
        with fits.open(mask_path) as hdul:
            data = hdul[1].data if len(hdul)>1 else hdul[0].data
            vec = np.array(data).astype(float).ravel()
        ns_mask = infer_nside(vec)
        mask_vec = vec if ns_mask==nside else hp.ud_grade(vec, nside, power=-2)
    except Exception as e:
        print("Mask not used:", e)
        mask_vec = None

# --- Load Galaxy Zoo ---
try:
    gz1 = pd.read_csv(gz1_path, compression="infer")
except Exception:
    gz1 = pd.read_csv(gz1_path)
ra_col, dec_col = detect_ra_dec_columns(gz1)
spins, keep = extract_spin_gz1(gz1, min_margin=MIN_MARGIN)
gz1 = gz1.loc[keep].copy(); spins = spins.loc[gz1.index]
ra_deg, dec_deg = parse_ra_dec_mixed(gz1[ra_col], gz1[dec_col])
ok = np.isfinite(ra_deg) & np.isfinite(dec_deg)
gz1 = gz1.loc[ok].copy(); spins = spins.loc[gz1.index]
ra_deg = ra_deg[ok]; dec_deg = dec_deg[ok]

# --- Geometry & pix (corrected: NEST=True) ---
thetas = (np.pi/2.0) - np.deg2rad(dec_deg)
phis   = np.deg2rad(ra_deg) % (2*np.pi)
pix = hp.ang2pix(nside, thetas, phis, nest=True)  # <— NEST=True per sweep

# --- Swap components and apply phi metric (div_sin) ---
# swap=True ⇒ use map’s dphi as our dtheta, and map’s dtheta as our dphi
dth_swap = dphi_map[pix].astype(float)
dph_swap = dtheta_map[pix].astype(float)
s = np.sin(thetas); s[s==0] = 1.0
dph_metric = dph_swap / s     # <— div_sin
sky_ok_vec = (mask_vec[pix] > 0) if mask_vec is not None else np.ones_like(dph_metric, dtype=bool)

# --- Run (optionally both with/without mask) ---
variants = [("unmasked", False)]
if EVALUATE_BOTH_MASK_SETTINGS:
    variants = [("unmasked", False), ("masked", True)]
else:
    variants = [("masked" if USE_MASK else "unmasked", USE_MASK)]

stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
base = f"/mnt/data/Mechanism1_OptionA_GZ1_CORRECTED_{stamp}"
os.makedirs(base, exist_ok=True)

summaries = []
for tag, use_mask in variants:
    sky_ok = sky_ok_vec if use_mask else np.ones_like(sky_ok_vec, dtype=bool)
    res = summarize(spins.values, dph_metric, dth_swap, THETA_GATE_DEG, sky_ok=sky_ok)
    if res["N"] == 0:
        print(f"{tag}: no valid samples"); continue

    # rotation nulls
    valid = res["valid"]
    rot_xs, rot_ys = rotation_nulls(dth_swap[valid], dph_metric[valid], ROTATIONS_DEG)
    rot_fracs=[]
    for y_rot in rot_ys:
        pred_rot = -np.where(y_rot>=0, 1, -1)  # keep handedness flip
        rot_fracs.append((spins.values[valid].astype(int) == pred_rot.astype(int)).mean())

    # shuffles
    N_valid = int(valid.sum())
    if N_valid > 250_000:
        idx = np.random.choice(np.where(valid)[0], size=250_000, replace=False)
        from_idx = spins.values[idx]; dph_idx = dph_metric[idx]; dth_idx = dth_swap[idx]
        # simple shuffle null
        fracs=[]
        for _ in range(MAX_SHUFFLE_N):
            np.random.shuffle(from_idx)
            fracs.append((from_idx == -np.where(dph_idx>=0, 1, -1)).mean())
        shuffle_mean, shuffle_std = float(np.mean(fracs)), float(np.std(fracs))
    else:
        fracs=[]
        arr = spins.values[valid].astype(int).copy()
        pred = -np.where(dph_metric[valid]>=0, 1, -1).astype(int)
        for _ in range(MAX_SHUFFLE_N):
            np.random.shuffle(arr)
            fracs.append((arr==pred).mean())
        shuffle_mean, shuffle_std = float(np.mean(fracs)), float(np.std(fracs))

    out_dir = os.path.join(base, tag); os.makedirs(out_dir, exist_ok=True)
    write_bundle(tag, valid, spins.values, res["pred"], dph_metric, dth_swap, out_dir)

    summary = dict(
        variant=tag,
        nside=int(nside),
        gate_deg=float(THETA_GATE_DEG),
        use_mask=bool(use_mask),
        N=int(res["N"]), k=int(res["k"]),
        frac=float(res["frac"]), ci_lo=float(res["ci_lo"]), ci_hi=float(res["ci_hi"]),
        rotation_nulls=dict(zip([str(x) for x in ROTATIONS_DEG], [float(x) for x in rot_fracs])),
        shuffle_null_mean=shuffle_mean, shuffle_null_std=shuffle_std,
        maps={"dtheta": dtheta_path, "dphi": dphi_path, "mask": mask_path},
        gz1_path=gz1_path,
        corrections={"nest": True, "swap_components": True, "phi_metric": "div_sin", "pred_sign": "-dphi"}
    )
    summaries.append(summary)
    with open(os.path.join(out_dir, "summary.json"), "w") as f:
        json.dump(summary, f, indent=2)
    with open(os.path.join(out_dir, "alignment_summary.txt"), "w") as f:
        f.write(
            f"Variant: {tag}\n"
            f"N (valid) = {summary['N']}\n"
            f"Alignment fraction = {summary['frac']:.6f}  (95% CI [{summary['ci_lo']:.6f}, {summary['ci_hi']:.6f}])\n"
            f"Rotation nulls: {summary['rotation_nulls']}\n"
            f"Shuffle mean±std = {shuffle_mean:.6f} ± {shuffle_std:.6f}\n"
        )

# write top-level summary
with open(os.path.join(base, "run_summaries.json"), "w") as f:
    json.dump(summaries, f, indent=2)

# zip & download
zip_path = f"{base}.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as zf:
    for root, _, files in os.walk(base):
        for fn in files:
            fp = os.path.join(root, fn)
            zf.write(fp, arcname=os.path.relpath(fp, base))

print("DONE.")
print("Results dir:", base)
print("ZIP:", zip_path)
try:
    from google.colab import files
    files.download(zip_path)
except Exception as e:
    print("Colab download not available:", e)


Inputs:
  dtheta: ./Logosfield_dtheta_map.npy
  dphi  : ./Logosfield_dphi_map.npy
  gz1   : ./GalaxyZoo1_DR_table2.csv.gz
  mask  : ./glimpse_mask.fits


/tmp/ipython-input-403693560.py:210: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


DONE.
Results dir: /mnt/data/Mechanism1_OptionA_GZ1_CORRECTED_20250831_012328
ZIP: /mnt/data/Mechanism1_OptionA_GZ1_CORRECTED_20250831_012328.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [41]:
def gradient_from_scalar(mapvec, nside, lmax=None):
    """
    Return (d/dθ, d/dφ) of a scalar HEALPix map.
    Tries sphtfunc.{alm2map_der, alm2map_der1}; falls back to a safe
    numeric interpolation if neither is available.
    """
    # 1) Harmonic path (preferred: exact & fast)
    try:
        alm = hp.sphtfunc.map2alm(mapvec, lmax=lmax)
        if hasattr(hp.sphtfunc, "alm2map_der"):
            dth, dph = hp.sphtfunc.alm2map_der(alm, nside, lmax=lmax)
            return np.array(dth), np.array(dph)
        if hasattr(hp.sphtfunc, "alm2map_der1"):
            dth, dph = hp.sphtfunc.alm2map_der1(alm, nside, lmax=lmax)
            return np.array(dth), np.array(dph)
    except Exception:
        pass  # fall through to numeric

    # 2) Numeric fallback via spherical interpolation (slower, but robust)
    pix = np.arange(12 * (nside ** 2))
    theta, phi = hp.pix2ang(nside, pix, nest=False)  # mapvec is a plain HEALPix vector
    eps = 1e-3  # ~0.057°; small enough for local derivative

    # central differences in θ
    f_th_plus  = hp.get_interp_val(mapvec, theta + eps, phi, nest=False)
    f_th_minus = hp.get_interp_val(mapvec, theta - eps, phi, nest=False)
    dth = (f_th_plus - f_th_minus) / (2.0 * eps)

    # central differences in φ (wrap handled by get_interp_val)
    f_ph_plus  = hp.get_interp_val(mapvec, theta, phi + eps, nest=False)
    f_ph_minus = hp.get_interp_val(mapvec, theta, phi - eps, nest=False)
    dph = (f_ph_plus - f_ph_minus) / (2.0 * eps)

    return np.array(dth), np.array(dph)


In [43]:
# === All-in-one (v2): Mechanism-2 re-anchoring + Option-A (Galaxy Zoo) ===
# Patched: robust gradient_from_scalar (der → der1 → numeric fallback)

import os, sys, glob, json, math, zipfile, random
from datetime import datetime
import numpy as np, pandas as pd
import matplotlib.pyplot as plt

# Optional: uncomment to ensure harmonic derivative is available
# import sys, subprocess; subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "healpy>=1.16.0"])

THETA_GATE_DEG = 20.0
MIN_MARGIN = 0.05
ALPHA_DENS = 1.0
ALPHA_KAPPA = 1.0
SMOOTH_FWHM_DEG = 0.0
LMAX = None
USE_MASK_FOR_GZ1 = False
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED); random.seed(RANDOM_SEED)

def ensure_healpy_astropy():
    try:
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        from astropy.io import fits
        return hp, SkyCoord, u, fits
    except Exception:
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", "healpy", "astropy"])
        import healpy as hp
        from astropy.coordinates import SkyCoord
        from astropy import units as u
        from astropy.io import fits
        return hp, SkyCoord, u, fits
hp, SkyCoord, u, fits = ensure_healpy_astropy()

# ---------- Patched gradient ----------
def gradient_from_scalar(mapvec, nside, lmax=None):
    """
    Return (∂/∂θ, ∂/∂φ) of a scalar HEALPix map.
    Try harmonic route, else robust numeric fallback via interpolation.
    """
    # Harmonic (preferred)
    try:
        alm = hp.sphtfunc.map2alm(mapvec, lmax=lmax)
        if hasattr(hp.sphtfunc, "alm2map_der"):
            dth, dph = hp.sphtfunc.alm2map_der(alm, nside, lmax=lmax)
            return np.array(dth), np.array(dph)
        if hasattr(hp.sphtfunc, "alm2map_der1"):
            dth, dph = hp.sphtfunc.alm2map_der1(alm, nside, lmax=lmax)
            return np.array(dth), np.array(dph)
    except Exception:
        pass
    # Numeric fallback
    pix = np.arange(12*(nside**2))
    theta, phi = hp.pix2ang(nside, pix, nest=False)
    eps = 1e-3
    f_th_plus  = hp.get_interp_val(mapvec, theta + eps, phi, nest=False)
    f_th_minus = hp.get_interp_val(mapvec, theta - eps, phi, nest=False)
    dth = (f_th_plus - f_th_minus)/(2*eps)
    f_ph_plus  = hp.get_interp_val(mapvec, theta, phi + eps, nest=False)
    f_ph_minus = hp.get_interp_val(mapvec, theta, phi - eps, nest=False)
    dph = (f_ph_plus - f_ph_minus)/(2*eps)
    return np.array(dth), np.array(dph)

# ---------- Discovery ----------
def autodiscover(pats, roots=("/mnt/data","/content",".")):
    out=[]
    for r in roots:
        if not os.path.isdir(r): continue
        for p in pats: out += glob.glob(os.path.join(r,p))
    return sorted(set(out), key=lambda p:(len(os.path.basename(p)), p.lower()))
def discover_map(kind):
    for pats in [[f"*{kind}*.npy"], [f"*Logosfield*{kind}*.npy"], [f"*{kind}_map*.npy"], [f"*{kind} map*.npy"]]:
        c = autodiscover(pats)
        if c: return c[0]
    return None
def discover_overlay_candidates():
    dens = autodiscover(["*density*.npy","*overdensity*.npy","*rho*.npy","*scalar_density*.npy","*density*.npz"])
    kappa = autodiscover(["*kappa*.npy","*convergence*.npy","*mech2*.npy","*kappa*.npz"])
    return (dens[0] if dens else None), (kappa[0] if kappa else None)
def discover_gz1():
    c = autodiscover(["GalaxyZoo*table2*.csv*","*GZ*table2*.csv*","*galaxy*zoo*table2*.csv*"])
    return c[0] if c else None
def discover_jwst():
    c = autodiscover(["*STANDARDIZED*GOODS*.xlsx","*highz*.xlsx","*JWST*.xlsx","*GOODS*.xlsx"])
    return c[0] if c else None
def discover_mask():
    c = autodiscover(["*mask*.fits"])
    return c[0] if c else None

# ---------- Utilities ----------
def infer_nside(vec):
    n=int(vec.size); ns=int(round((n/12.0)**0.5))
    if 12*(ns**2)!=n: raise ValueError(f"Length {n} != 12*nside^2")
    return ns
def detect_ra_dec_columns(df):
    L={c.lower():c for c in df.columns}
    ra  = L.get("ra") or L.get("ra_deg") or L.get("ra (deg)") or next((c for c in df.columns if "ra" in c.lower()),None)
    dec = L.get("dec") or L.get("dec_deg") or L.get("dec (deg)") or L.get("de") or next((c for c in df.columns if "dec" in c.lower() or c.lower()=="de"),None)
    if ra is None or dec is None: raise ValueError("RA/Dec columns not found.")
    return ra, dec
def parse_ra_dec_mixed(ra_s, dec_s):
    ra_num=pd.to_numeric(ra_s, errors="coerce"); dec_num=pd.to_numeric(dec_s, errors="coerce")
    if ra_num.notna().all() and dec_num.notna().all():
        return ra_num.values.astype(float), dec_num.values.astype(float)
    sc=SkyCoord(ra=ra_s.astype(str).values, dec=dec_s.astype(str).values, unit=(u.hourangle,u.deg), frame="icrs")
    return sc.ra.deg.astype(float), sc.dec.deg.astype(float)
def extract_spin_gz1(df, min_margin=0.05):
    p_cw=next((c for c in df.columns if c.lower() in ("p_cw","p(cw)","p_cw_prob","prob_cw")),None)
    p_acw=next((c for c in df.columns if c.lower() in ("p_acw","p(ccw)","p_acw_prob","prob_acw","p_ccw","prob_ccw")),None)
    if p_cw and p_acw:
        pc=pd.to_numeric(df[p_cw],errors="coerce").astype(float)
        pa=pd.to_numeric(df[p_acw],errors="coerce").astype(float)
        keep=(pc-pa).abs()>=min_margin
        spin=np.where(pc>pa,1,-1).astype(int)
        return pd.Series(spin,index=df.index), keep
    for c in df.columns:
        if c.lower() in ("spin","handedness","spiral","cw_ccw"):
            v=df[c].astype(str).str.lower().str.strip()
            spin=np.where(v.isin(["cw","+1","1","clockwise"]),1,
                          np.where(v.isin(["ccw","-1","counterclockwise","anticlockwise"]),-1,np.nan))
            keep=~np.isnan(spin); return pd.Series(spin,index=df.index).astype(int), keep
    raise ValueError("No spin columns found.")
def phi_metric_divsin(dph, thetas):
    s=np.sin(thetas).copy(); s[s==0]=1.0
    return dph/s
def smooth_if_needed(mapvec, fwhm_deg):
    if not fwhm_deg or fwhm_deg<=0: return mapvec
    return hp.sphtfunc.smoothing(mapvec, fwhm=math.radians(fwhm_deg), verbose=False)
def fit_linear_M(Lx,Ly,Ox,Oy):
    A1=np.column_stack([Lx,Ly,np.zeros_like(Lx),np.zeros_like(Lx)])
    A2=np.column_stack([np.zeros_like(Lx),np.zeros_like(Lx),Lx,Ly])
    A=np.vstack([A1,A2]); b=np.concatenate([Ox,Oy])
    lam=1e-6; ATA=A.T@A+lam*np.eye(4); ATb=A.T@b
    m=np.linalg.solve(ATA,ATb); return np.array([[m[0],m[1]],[m[2],m[3]]])
def optionA_stats(spins, dph_metric, dth, thetas, gate_deg, sky_ok=None):
    pred = -np.where(dph_metric>=0,1,-1)
    alpha=np.arctan2(dth,dph_metric)
    gate=np.abs(alpha)<=np.deg2rad(gate_deg)
    valid=gate & np.isfinite(dph_metric) & np.isfinite(dth)
    if sky_ok is not None: valid&=sky_ok
    N=int(valid.sum())
    if N==0: return dict(N=0, frac=np.nan, ci_lo=np.nan, ci_hi=np.nan, valid=valid, pred=pred)
    k=int((spins[valid].astype(int)==pred[valid].astype(int)).sum())
    frac=k/N; se=(frac*(1-frac)/N)**0.5; lo=max(0.0,frac-1.96*se); hi=min(1.0,frac+1.96*se)
    return dict(N=N,k=k,frac=frac,ci_lo=lo,ci_hi=hi,valid=valid,pred=pred)

def load_scalar_map_any(path, target_nside=None, name_hint=""):
    if path is None or not os.path.exists(path): return None
    arr=np.load(path, allow_pickle=True)
    if isinstance(arr, np.lib.npyio.NpzFile):
        arr=arr[list(arr.keys())[0]]
    if isinstance(arr,np.ndarray) and arr.dtype==object:
        try:
            obj=arr.item() if arr.size==1 else arr[0]
            if isinstance(obj,dict):
                for k in ["map","data","values","density","arr","field"]:
                    if k in obj: arr=np.asarray(obj[k]); break
                else: arr=np.asarray(next(iter(obj.values())))
            else: arr=np.asarray(obj)
        except Exception:
            arr=np.asarray(arr)
    arr=np.asarray(arr,dtype=float).ravel()
    src_nside=infer_nside(arr)
    if target_nside is not None and src_nside!=target_nside:
        arr=hp.ud_grade(arr, target_nside, power=0)
    return arr

# ---------- Load inputs ----------
dtheta_path=discover_map("dtheta")
dphi_path  =discover_map("dphi")
gz1_path   =discover_gz1()
jwst_path  =discover_jwst()
mask_path  =discover_mask()
dens_path,kappa_path=discover_overlay_candidates()

print("Inputs:")
print("  dtheta:", dtheta_path)
print("  dphi  :", dphi_path)
print("  gz1   :", gz1_path)
print("  jwst  :", jwst_path or "(not found; all-sky calibration)")
print("  mask  :", mask_path or "(none)")
print("  density overlay:", dens_path or "(none)")
print("  kappa overlay  :", kappa_path or "(none)")

dtheta_map=np.load(dtheta_path)
dphi_map  =np.load(dphi_path)
nside=infer_nside(dtheta_map); assert infer_nside(dphi_map)==nside

dens_map=load_scalar_map_any(dens_path,  target_nside=nside, name_hint="density")
kmap    =load_scalar_map_any(kappa_path, target_nside=nside, name_hint="kappa")

mask_vec=None
if mask_path:
    try:
        with fits.open(mask_path) as hdul:
            data=hdul[1].data if len(hdul)>1 else hdul[0].data
            vec=np.array(data).astype(float).ravel()
        ns_mask=infer_nside(vec)
        mask_vec=vec if ns_mask==nside else hp.ud_grade(vec, nside, power=0)
    except Exception as e:
        print("Mask not used:", e); mask_vec=None

# ---------- Mechanism-2 composite & gradient ----------
mech2_comp=None
if (dens_map is not None) or (kmap is not None):
    parts=[]
    if dens_map is not None:
        dm=(dens_map-np.nanmean(dens_map))/(np.nanstd(dens_map)+1e-12); parts.append(ALPHA_DENS*dm)
    if kmap is not None:
        km=(kmap-np.nanmean(kmap))/(np.nanstd(kmap)+1e-12); parts.append(ALPHA_KAPPA*km)
    mech2_comp=np.sum(parts,axis=0); mech2_comp=smooth_if_needed(mech2_comp, SMOOTH_FWHM_DEG)
    dth_M2, dph_M2 = gradient_from_scalar(mech2_comp, nside, lmax=LMAX)
else:
    dth_M2, dph_M2 = None, None

# ---------- Calibration set ----------
cal_pix=None
if jwst_path and os.path.exists(jwst_path):
    try:
        jw=pd.read_excel(jwst_path)
        ra_j,dec_j=detect_ra_dec_columns(jw)
        ra_deg_j,dec_deg_j=parse_ra_dec_mixed(jw[ra_j],jw[dec_j])
        th_j=(np.pi/2.0)-np.deg2rad(dec_deg_j); ph_j=np.deg2rad(ra_deg_j)%(2*np.pi)
        cal_pix=hp.ang2pix(nside, th_j, ph_j, nest=True)
        zcol=next((c for c in jw.columns if c.lower() in ("z","redshift","photoz","z_phot","z_spec")),None)
        if zcol is not None:
            z=pd.to_numeric(jw[zcol],errors="coerce")
            if np.isfinite(z).any():
                highz = z >= np.nanmedian(z)
                if len(highz)==len(cal_pix): cal_pix=cal_pix[highz.values]
    except Exception as e:
        print("JWST calibration fallback (all-sky):", e); cal_pix=None
if cal_pix is None: cal_pix=np.arange(12*(nside**2))

# ---------- Logosfield gradients with corrections ----------
pix_all=np.arange(12*(nside**2))
th_all, ph_all = hp.pix2ang(nside, pix_all, nest=True)
dthL_all = dphi_map[pix_all].astype(float)   # swap
dphL_all = dtheta_map[pix_all].astype(float) # swap

M=np.eye(2); cal_report={}
if (dth_M2 is not None) and (dph_M2 is not None):
    sel=np.unique(cal_pix)
    ok=np.isfinite(dthL_all[sel])&np.isfinite(dphL_all[sel])&np.isfinite(dth_M2[sel])&np.isfinite(dph_M2[sel])
    idx=sel[ok]
    if idx.size>=100:
        M=fit_linear_M(dthL_all[idx], dphL_all[idx], dth_M2[idx], dph_M2[idx])
        predx=M[0,0]*dthL_all[idx]+M[0,1]*dphL_all[idx]
        predy=M[1,0]*dthL_all[idx]+M[1,1]*dphL_all[idx]
        cal_report={"num_cal_pix":int(idx.size),
                    "M":M.tolist(),
                    "corr_x":float(np.corrcoef(predx,dth_M2[idx])[0,1]),
                    "corr_y":float(np.corrcoef(predy,dph_M2[idx])[0,1])}
    else:
        print("Not enough calibration pixels; using identity M.")

dthL_adj_all = M[0,0]*dthL_all + M[0,1]*dphL_all
dphL_adj_all = M[1,0]*dthL_all + M[1,1]*dphL_all

# ---------- Galaxy Zoo Option-A ----------
try:
    gz1=pd.read_csv(gz1_path, compression="infer")
except Exception:
    gz1=pd.read_csv(gz1_path)
ra_col,dec_col=detect_ra_dec_columns(gz1)
spins,keep=extract_spin_gz1(gz1, MIN_MARGIN)
gz1=gz1.loc[keep].copy(); spins=spins.loc[gz1.index]
ra_deg,dec_deg=parse_ra_dec_mixed(gz1[ra_col],gz1[dec_col])
ok=np.isfinite(ra_deg)&np.isfinite(dec_deg)
gz1=gz1.loc[ok].copy(); spins=spins.loc[gz1.index]
ra_deg=ra_deg[ok]; dec_deg=dec_deg[ok]

thetas=(np.pi/2.0)-np.deg2rad(dec_deg); phis=np.deg2rad(ra_deg)%(2*np.pi)
pix_gz=hp.ang2pix(nside, thetas, phis, nest=True)

dth_gz=dthL_adj_all[pix_gz]
dph_gz=dphL_adj_all[pix_gz]
dph_metric=phi_metric_divsin(dph_gz, thetas)

sky_ok=np.ones_like(dph_gz, bool)
if USE_MASK_FOR_GZ1 and ('mask_vec' in locals()) and (mask_vec is not None):
    sky_ok = (mask_vec[pix_gz] > 0)

stats=optionA_stats(spins.values, dph_metric, dth_gz, thetas, THETA_GATE_DEG, sky_ok=sky_ok)

# ---------- Save & download ----------
stamp=datetime.utcnow().strftime("%Y%m%d_%H%M%S")
out_dir=f"/mnt/data/Step3_AllInOne_GZ1_v2_{stamp}"
os.makedirs(out_dir, exist_ok=True)
summary={
    "nside":int(nside),
    "theta_gate_deg":THETA_GATE_DEG,
    "min_margin":MIN_MARGIN,
    "use_mask":bool(USE_MASK_FOR_GZ1),
    "inputs":{"dtheta":dtheta_path,"dphi":dphi_path,"gz1":gz1_path,"jwst_for_cal":jwst_path,
              "density":dens_path,"kappa":kappa_path,"mask":mask_path},
    "mechanism2":{"alpha_density":ALPHA_DENS,"alpha_kappa":ALPHA_KAPPA,
                  "smooth_fwhm_deg":SMOOTH_FWHM_DEG,"lmax":LMAX,
                  "available": bool((dens_map is not None) or (kmap is not None))},
    "calibration":cal_report,
    "transform_M":M.tolist(),
    "optionA_result":{k:(float(v) if isinstance(v,(int,float,np.floating)) else v)
                      for k,v in stats.items() if k not in ("valid","pred")},
}
with open(os.path.join(out_dir,"summary.json"),"w") as f: json.dump(summary,f,indent=2)

pd.DataFrame({
    "valid":stats["valid"].astype(int),
    "spin_obs":spins.values.astype(int),
    "pred_spin":stats["pred"].astype(int),
    "dtheta_reanchored":dth_gz.astype(float),
    "dphi_metric_reanchored":dph_metric.astype(float),
}).to_csv(os.path.join(out_dir,"gz1_objects_reanchored.csv"), index=False)

plt.figure(figsize=(6,4))
plt.title("Galaxy Zoo — Option A (Re-anchored by Mechanism-2)")
plt.axhline(0.5, ls="--"); plt.bar(["Observed"], [stats["frac"] if stats["N"]>0 else 0.0])
plt.ylabel("Alignment fraction"); plt.ylim(0.45,0.75); plt.tight_layout()
plt.savefig(os.path.join(out_dir,"plot_alignment_reanchored.png"), dpi=160); plt.close()

if (dens_map is not None) or (kmap is not None):
    if 'mech2_comp' in locals() and mech2_comp is not None:
        np.save(os.path.join(out_dir,"mech2_composite.npy"), mech2_comp.astype(np.float32))

zip_path=f"{out_dir}.zip"
with zipfile.ZipFile(zip_path,"w",compression=zipfile.ZIP_DEFLATED) as zf:
    for root,_,files in os.walk(out_dir):
        for fn in files: zf.write(os.path.join(root,fn), arcname=os.path.relpath(os.path.join(root,fn), out_dir))

print("DONE."); print("Results dir:", out_dir); print("ZIP:", zip_path)
try:
    from google.colab import files; files.download(zip_path)
except Exception as e:
    print("Colab download not available:", e)


Inputs:
  dtheta: ./Logosfield_dtheta_map.npy
  dphi  : ./Logosfield_dphi_map.npy
  gz1   : ./GalaxyZoo1_DR_table2.csv.gz
  jwst  : ./master_highz_plus_goodsn_filled copy (1)_STANDARDIZED (1) (1).xlsx
  mask  : ./glimpse_mask.fits
  density overlay: ./Logosfield_scalar_density_map.npy
  kappa overlay  : (none)
Not enough calibration pixels; using identity M.


/tmp/ipython-input-241384773.py:291: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  stamp=datetime.utcnow().strftime("%Y%m%d_%H%M%S")


DONE.
Results dir: /mnt/data/Step3_AllInOne_GZ1_v2_20250831_014834
ZIP: /mnt/data/Step3_AllInOne_GZ1_v2_20250831_014834.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>